# Classifieur thermique

On mets en place le classifieur prenant en compte que les images thermiques.

## Choix et calcul des features

Dépendances


In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os

df = pd.read_csv('base_de_données.csv')
df_train = pd.read_csv('X_train.csv')
df_test = pd.read_csv('X_test.csv')
csv_thermiques_train = df_train['nom_fichier']
indices_train = df_train['index']
csv_thermiques_test = df_test['nom_fichier']
indices_test = df_test['index']
csv_thermiques = df['nom_fichier']
csv = csv_thermiques[0]
csv_0 = (csv.split('.'))[0]
sift = cv2.SIFT_create()

Création des images et des arrays min et max

In [2]:
# arr_min = []
# arr_max = []
# count=1
# for csv in csv_thermiques:
# 	nom_csv = (csv.split('.'))[0]
# 	data = pd.read_csv('./thermiques/' + csv ,delimiter=';')
# 	img = data.to_numpy()
# 	#shape (287,383)
# 	img = img[:,:-1]
# 	#shape (287,382)
# 	for i in range(img.shape[0]):
# 		for j in range(img.shape[1]):
# 			L = img[i,j].split(',')
# 			if len(L) == 2:
# 				img[i,j] = L[0] + '.' + L[1]
# 			else:
# 				img[i,j] = L[0] + '.' + '0'

# 	img = img.astype(np.float64)
# 	mask = (img > 60)
	
# 	min = np.min(img[mask])
# 	max = np.max(img[mask])
# 	img = img - min
# 	img = ( img / (max-min) ) * 255

# 	image = mask*img
# 	image = np.round(image,decimals=0)
# 	image = image.astype(np.uint8)
# 	mask = np.array(np.uint8(mask))
# 	cv2.imwrite('./cv2_thermiques/' + nom_csv + '.png',image)
# 	np.save('./mask_thermique/' + nom_csv + '.npy',mask)
# 	arr_min.append(min)
# 	arr_max.append(max)
# 	print(count)
# 	count += 1
	

	



In [3]:
# a_min = np.array(arr_min)
# a_max = np.array(arr_max)

# np.save('./array_min_max/array_min.npy',a_min)
# np.save('./array_min_max/array_max.npy',a_max)


Création des descripteurs

In [4]:
i=0
for index,csv in zip(indices_train,csv_thermiques_train) :
	nom_csv = (csv.split('.'))[0]
	if i==0:
		img = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
		mask = np.load('./mask_thermique/'+ nom_csv + '.npy')
		
		#kp,des = sift.detectAndCompute(img,None)
		kp = sift.detect(img,mask = mask)
		kp,des = sift.compute(img,kp)
		descripteurs = des	
	else:
		img = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
		mask = np.load('./mask_thermique/'+ nom_csv + '.npy')
		
		#kp,des = sift.detectAndCompute(img,None)
		kp = sift.detect(img,mask = mask)
		kp,des = sift.compute(img,kp)		
		descripteurs = np.concatenate((descripteurs,des),axis=0)
	i+=1
	print(i)

descripteurs = np.float32(descripteurs)
print(descripteurs.shape) #(22705,128)
np.save('descripteurs_th.npy',descripteurs)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
(1615, 128)


Création des labels

In [5]:
#Creation des labels (tache : non conforme (1) si q_tache>3)

q_tache_train = df_train['q_tache'].fillna(0)
Y_train = q_tache_train.to_numpy(np.float32)
Y_train = (Y_train >3)
Y_train = np.int16(Y_train)

q_tache_test = df_test['q_tache'].fillna(0)
Y_test = q_tache_test.to_numpy(np.float32)
Y_test = (Y_test >3)
Y_test = np.int16(Y_test)


np.save('Y_train.npy',Y_train)
np.save('Y_test.npy',Y_test)

Création des histogrammes sur les jeux

In [6]:
descripteurs = np.load('descripteurs_th.npy')

#Calculs des center des k means
nbre_clusters = 5 #tres petit nombre car faible nombre de features de base sur les img thermiques

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center=cv2.kmeans(descripteurs,nbre_clusters,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
np.save('center_k_means_th.npy',center)

Y_train = np.load('Y_train.npy')
n = Y_train.shape[0]
X_train = np.zeros((n,nbre_clusters+2))

arr_min = np.load('./array_min_max/array_min.npy')
arr_max = np.load('./array_min_max/array_max.npy')

center_k_means = np.load('center_k_means_th.npy')

k=0
for indice,csv in zip(indices_train , csv_thermiques_train) :
	nom_csv = (csv.split('.'))[0]
	
	img = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
	mask = np.load('./mask_thermique/'+ nom_csv + '.npy')
		
	#kp,des = sift.detectAndCompute(img,None)
	kp = sift.detect(img,mask = mask)
	kp,des = sift.compute(img,kp)
	nbr_des = des.shape[0]
	x = np.zeros((nbre_clusters+2,))
	for elem in des:
		distance = np.linalg.norm(center_k_means - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	x[nbre_clusters] = arr_min[indice]
	x[nbre_clusters+1] = arr_max[indice]
	X_train[k] = x
	k+=1
	print(k)

np.save('X_train.npy',X_train)
print(X_train.shape)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
(95, 7)


In [7]:
m = len(csv_thermiques)-X_train.shape[0]
X_test = np.zeros((m,nbre_clusters+2))
print(m)


k=0
for indice,csv in zip(indices_test , csv_thermiques_test) :
	
	nom_csv = (csv.split('.'))[0]
	img = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
	mask = np.load('./mask_thermique/'+ nom_csv + '.npy')
	
	#kp,des = sift.detectAndCompute(img,None)
	kp = sift.detect(img,mask = mask)
	kp,des = sift.compute(img,kp)
	nbr_des = des.shape[0]
	x = np.zeros((nbre_clusters+2,))
	for elem in des:
		distance = np.linalg.norm(center_k_means - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	x[nbre_clusters] = arr_min[indice]
	x[nbre_clusters+1] = arr_max[indice]
	X_test[k] = x
	k+=1
	print(k)
	
np.save('X_test.npy',X_test)

84
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


Classifieurs  
LDA

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis()

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')


LDA.fit(X_train,Y_train)
Y_pred = LDA.predict(X_test)

def matrice_confusion(pred,lab):
	#calcul des vrais positifs
	vp = np.count_nonzero((pred == 1) & (lab == 1))
	#calculs des faux positifs
	fp = np.count_nonzero((pred == 1) & (lab == 0))
	#calculs des vrais négatifs
	vn = np.count_nonzero((pred == 0) & (lab == 0))
	#calculs des faux négatifs
	fn = np.count_nonzero((pred == 0) & (lab == 1))
	return [vp,fp,vn,fn]
	
conf = matrice_confusion(Y_pred,Y_test)
print(conf)
taux_fn = conf[3]/sum(conf)
taux_fn_neg = conf[3]/(conf[3]+conf[0])
non_conf = conf[0]+conf[3]
conf = conf[1]+conf[2]

score_random = max(conf/(conf+non_conf),non_conf/(conf+non_conf))

print(f"Non conformes : {non_conf}")
print(f"Conformes : {conf}")
print(f"taux de faux conformes par rapport aux non conformes : {taux_fn_neg}")

print(f"score = {LDA.score(X_test,Y_test)}")
print(f"score du random = {score_random}")



[35, 21, 9, 19]
Non conformes : 54
Conformes : 30
taux de faux conformes par rapport aux non conformes : 0.35185185185185186
score = 0.5238095238095238
score du random = 0.6428571428571429


1-NN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

neigh.fit(X_train,Y_train)
Y_pred = neigh.predict(X_test)

conf = matrice_confusion(Y_pred,Y_test)
print(conf)
taux_fn = conf[3]/sum(conf)
taux_fn_neg = conf[3]/(conf[3]+conf[0])
non_conf = conf[0]+conf[3]
conf = conf[1]+conf[2]

score_random = max(conf/(conf+non_conf),non_conf/(conf+non_conf))

print(f"Non conformes : {non_conf}")
print(f"Conformes : {conf}")
print(f"taux de faux conformes par rapport aux non conformes : {taux_fn_neg}")

print(f"score = {neigh.score(X_test,Y_test)}")
print(f"score du random = {score_random}")

[40, 11, 19, 14]
Non conformes : 54
Conformes : 30
taux de faux conformes par rapport aux non conformes : 0.25925925925925924
score = 0.7023809523809523
score du random = 0.6428571428571429


score:  

1-NN : 0.88  
2-NN : 0.80    
3-NN : 0.79  
4-NN : 0.80  
5-NN : 0.80